### 11주차 블록체인프로그래밍 과제 
#### 휴먼지능정보공학과 201810756 김대환

In [213]:
%%writefile src/Bank11.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.12;
contract Bank11{
    address owner;
    uint balance;
    uint tmp;
    event PrintLog(address from, uint amount);
    constructor(){
        owner = msg.sender;
        balance = address(this).balance;
    }
    fallback () external{
        emit PrintLog(owner,333);
    }
    function deposit(uint amount) payable public{
        require(msg.value == amount);
        balance += amount;
    }
    function withdrawAll() public{
        balance -= address(this).balance;
        payable(owner).transfer(address(this).balance);
    }
    function getBalance() public view returns(uint, uint){
        return (address(this).balance, balance);
    }
    function forwardTo(address payable _receiver, uint amount) public payable{
        require(_receiver.send(amount));
        balance -= amount;
    }
    function getAddrBalance(address addr) public view returns(uint){
        return addr.balance;
    }
}

Overwriting src/Bank11.sol


> 특정 계정의 잔고를 반환하는 getAddrBalance 함수도 구현하였지만(2번째 계정 등 특정 계정의 잔고를 반환하기 위해)\
기존 과제 지시사항에는 그런 함수에 대한 언급이 없어 사용하지는 않았습니다.

In [214]:
!cat src/Bank11.sol

//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.8.12;
contract Bank11{
    address owner;
    uint balance;
    uint tmp;
    event PrintLog(address from, uint amount);
    constructor(){
        owner = msg.sender;
        balance = address(this).balance;
    }
    fallback () external{
        emit PrintLog(owner,333);
    }
    function deposit(uint amount) payable public{
        require(msg.value == amount);
        balance += amount;
    }
    function withdrawAll() public{
        balance -= address(this).balance;
        payable(owner).transfer(address(this).balance);
    }
    function getBalance() public view returns(uint, uint){
        return (address(this).balance, balance);
    }
    function forwardTo(address payable _receiver, uint amount) public payable{
        require(_receiver.send(amount));
        balance -= amount;
    }
    function getAddrBalance(address addr) public view returns(uint){
        return addr.balance;
    }
}


In [215]:
!solc --optimize --combined-json abi,bin src/Bank11.sol > src/Bank11.json

In [216]:
!cat src/Bank11.json

{"contracts":{"src/Bank11.sol:Bank11":{"abi":[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"from","type":"address"},{"indexed":false,"internalType":"uint256","name":"amount","type":"uint256"}],"name":"PrintLog","type":"event"},{"stateMutability":"nonpayable","type":"fallback"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"deposit","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address payable","name":"_receiver","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"forwardTo","outputs":[],"stateMutability":"payable","type":"function"},{"inputs":[{"internalType":"address","name":"addr","type":"address"}],"name":"getAddrBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"getBalance","outputs":[{"in

#### 배포하기

In [217]:
%%writefile src/Bank11Deploy.js
var Web3 = require('web3');
var _abiBinJson = require('./Bank11.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
contractName=Object.keys(_abiBinJson.contracts);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(JSON.stringify(_abi));
_bin=_abiBinJson.contracts[contractName].bin;
async function deploy(){
    const accounts = await web3.eth.getAccounts();
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data : "0x"+_bin})
        .send({from: accounts[0], gas: 259210}, function(err,transactionHash){
            if(!err) console.log("hash: "+transactionHash);
        })
    console.log(deployed.options.address)
}
deploy()

Overwriting src/Bank11Deploy.js


In [80]:
!node src/Bank11Deploy.js

hash: 0x407e9b8e86ac28fdff6a5bcf026b4dadc3e10d6bd450e0d5293e0e1b2a6c9902
0x78207dD3f7a9B212B89d8Fc80Ed679B2b15D3311


In [81]:
%%writefile src/Bank11_1.js
var Web3 = require('web3');
var _abiBinJson = require('./Bank11.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
contractName=Object.keys(_abiBinJson.contracts);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(JSON.stringify(_abi));
var Bank = new web3.eth.Contract(_abiArray,'0x78207dD3f7a9B212B89d8Fc80Ed679B2b15D3311');
async function use(){
    const accounts = await web3.eth.getAccounts();
    await Bank.methods.deposit(11111).send({from:accounts[0],gas:259210,value:11111});
    await Bank.methods.deposit(222).send({from:accounts[0],gas:259210,value:222});
    await Bank.methods.getBalance().call().then(function(value){
        console.log("\t(1) contract balance : ",value)
    });
}
use()

Overwriting src/Bank11_1.js


##### - 입금 11111 wei, 222wei 하고 컨트랙 잔고 11333 wei 출력

In [82]:
!node src/Bank11_1.js

	(1) contract balance :  Result { '0': '11333', '1': '11333' }


In [83]:
%%writefile src/Bank11_2.js
var Web3 = require('web3');
var _abiBinJson = require('./Bank11.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
contractName=Object.keys(_abiBinJson.contracts);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(JSON.stringify(_abi));
var Bank = new web3.eth.Contract(_abiArray,'0x78207dD3f7a9B212B89d8Fc80Ed679B2b15D3311');
async function use(){
    const accounts = await web3.eth.getAccounts();
    const beforeBal = await web3.eth.getBalance(accounts[1]);
    console.log("\t (2) accounts[1] balance(before transfer) : ",beforeBal);
    await Bank.methods.forwardTo(accounts[1],333).send({from:accounts[0],gas:259210});
    await Bank.methods.getBalance().call().then(function(value){
        console.log("\t (2) contract balance(after transfer) : ",value)
    });
    const afterBal = await web3.eth.getBalance(accounts[1]);
    console.log("\t (2) accounts[1] balance(after transfer) : ",afterBal);
}
use()

Overwriting src/Bank11_2.js


##### - 자신의 2번째 계정으로 계좌이체 333 wei 하고 컨트랙 잔고 11000 wei 출력, 자신의 2 번째 계정 잔고 증가분(+333 wei) 출력

In [84]:
!node src/Bank11_2.js

	 (2) accounts[1] balance(before transfer) :  100000000000000003996
	 (2) contract balance(after transfer) :  Result { '0': '11000', '1': '11000' }
	 (2) accounts[1] balance(after transfer) :  100000000000000004329


> 계정의 증가분 333 wei는 제대로 계산되지 않았습니다. 단순히 값의 차이로 계산하니 계속해서 '0' 값만 출력됩니다.\
> 출력된 결과로는 333 wei 만큼 증가된 것을 확인할 수 있습니다.

In [85]:
%%writefile src/Bank11_3.js
var Web3 = require('web3');
var _abiBinJson = require('./Bank11.json');
var web3 = new Web3(new Web3.providers.HttpProvider("http://127.0.0.1:8345"));
contractName=Object.keys(_abiBinJson.contracts);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(JSON.stringify(_abi));
var Bank = new web3.eth.Contract(_abiArray,'0x78207dD3f7a9B212B89d8Fc80Ed679B2b15D3311');
async function use(){
    const accounts = await web3.eth.getAccounts();
    const beforeBal = await web3.eth.getBalance(accounts[0]);
    console.log("\t (3) accounts[0] balance(before withdrawAll) : ", beforeBal);
    await Bank.methods.withdrawAll().send({from:accounts[0],gas:259210});
    const afterBal = await web3.eth.getBalance(accounts[0]);
    console.log("\t (3) accounts[0] balance(after withdrawAll) : ", afterBal);
    await Bank.methods.getBalance().call().then(function(value){
        console.log("\t (3) contract balance(after withdrawAll) : ",value)
    });
}
use()

Overwriting src/Bank11_3.js


##### - 전액 인출하고 11000 wei, 자신의 잔고 증가분 출력 (+11000 wei)

In [86]:
!node src/Bank11_3.js

	 (3) accounts[0] balance(before withdrawAll) :  99940750099999971673
	 (3) accounts[0] balance(after withdrawAll) :  99940341639999982673
	 (3) contract balance(after withdrawAll) :  Result { '0': '0', '1': '0' }


> gas 비로 인해 차이가 11000 wei 만큼 출력되지는 않습니다.\
> 전액 인출 후 컨트랙 잔고가 0 인 것을 확인하였습니다.

In [105]:
%%writefile src/Bank11_4.js
var Web3 = require('web3');
var _abiBinJson = require('./Bank11.json');
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://127.0.0.1:8345"));
contractName=Object.keys(_abiBinJson.contracts);
_abi=_abiBinJson.contracts[contractName].abi;
_abiArray=JSON.parse(JSON.stringify(_abi));
var Bank = new web3.eth.Contract(_abiArray,'0x78207dD3f7a9B212B89d8Fc80Ed679B2b15D3311');
async function use(){
    const accounts = await web3.eth.getAccounts();
    var event = await Bank.events.PrintLog(function(error,result){
        if (!error) {
            console.log("Event fired: " + JSON.stringify(result) + "\n---> " + JSON.stringify(result.returnValues));
            Bank.methods.deposit(333).send({from:accounts[0],gas:259210,value:333});
        }
    });
    await web3.eth.sendTransaction({from:accounts[0],to:'0x78207dD3f7a9B212B89d8Fc80Ed679B2b15D3311'});
    await Bank.methods.getBalance().call().then(function(value){
        console.log("\t(4) contract balance(by fallback) : ",value)
    });
    process.exit(1);
}
use()

Overwriting src/Bank11_4.js


##### 컨트랙으로 111 송금하고, deposit() 이 아니라 fallback으로 입금하세요. 컨트랙 잔고 111 wei 출력

In [107]:
!node src/Bank11_4.js

Event fired: {"logIndex":0,"transactionIndex":0,"transactionHash":"0x4faffd59851e3bff0ab6a9c12611791384d83322209d1f449a4b7468bacd9a7e","blockHash":"0x3457f468e554939e4439c8005c9a9f8bc0484beb9a17d048163dd1a1151315b8","blockNumber":90,"address":"0x78207dD3f7a9B212B89d8Fc80Ed679B2b15D3311","type":"mined","removed":false,"id":"log_a0c4ebf8","returnValues":{"0":"0xFDD87F0271dB204Ab51C23eF4AB0323Dddc0B4cf","1":"333","from":"0xFDD87F0271dB204Ab51C23eF4AB0323Dddc0B4cf","amount":"333"},"event":"PrintLog","signature":"0xac247564f70be8d5e06a6068acf596e7bbca33994b256697b38d49095d4f3805","raw":{"data":"0x000000000000000000000000fdd87f0271db204ab51c23ef4ab0323dddc0b4cf000000000000000000000000000000000000000000000000000000000000014d","topics":["0xac247564f70be8d5e06a6068acf596e7bbca33994b256697b38d49095d4f3805"]}}
---> {"0":"0xFDD87F0271dB204Ab51C23eF4AB0323Dddc0B4cf","1":"333","from":"0xFDD87F0271dB204Ab51C23eF4AB0323Dddc0B4cf","amount":"333"}
	(4) contract balance(by fallback) :  Result { '0': '333

> 문제를 정확히 이해했는지 잘 모르겠습니다.\
fallback 을 일부러 호출하여 event에서 depsoit() 함수를 통해 컨트랙으로 입금하라는 것으로 이해하고\
그렇게 진행하였습니다.